In [1]:
import openslide as os
import pandas as pd
import numpy as np
import math
import cv2
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET
import skimage.morphology as sk_morphology
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
from multiprocessing import Pool, cpu_count
from pathlib import Path
import glob

In [3]:
import logging
import sys

def prepare_logger(logger, level=logging.DEBUG, filename_template="logs/notebook_log_{}.txt"):
    def prepare_handler(handler):
        handler.setLevel(level)
        handler.setFormatter(formatter)
        return handler

    formatter = logging.Formatter(fmt='%(asctime)s | [%(levelname)s] %(message)s',
                                 datefmt='%Y-%m-%d %H:%M:%S')

    
    del logger.handlers[:]
    logger.handlers.append(prepare_handler(logging.StreamHandler(sys.stderr)))
    
    if filename_template:
        path_file = filename_template.format(datetime.datetime.now().isoformat())
        logger.handlers.append(prepare_handler(logging.FileHandler(path_file)))

    logger.setLevel(level=level)
    return logger

In [7]:
def filter_background_camelyon(mini_slide):
    """ 
        Camelyon style background filter.
        Originally, the function should be applied on each tile individually. This, however, requires
        that each tile be extracted and read from the slide.
    """
    hsv_img = rgb2hsv(mini_slide.convert('RGB'))
    blurred_hsv_img = gaussian(hsv_img, multichannel=True)
    hsv_mask = Image.fromarray((blurred_hsv_img[:,:,1] >= 0.07).astype(np.uint8) * 255)
    return hsv_mask

In [8]:
""" EXPENSIVE FILTERS """

def mask_percent(np_img):
    """
        Calculates percentage of the masked area
    """
    if (len(np_img.shape) == 3) and (np_img.shape[2] == 3):
        np_sum = np_img[:, :, 0] + np_img[:, :, 1] + np_img[:, :, 2]
        mask_percentage = 100 - np.count_nonzero(np_sum) / np_sum.size * 100
    else:
        mask_percentage = 100 - np.count_nonzero(np_img) / np_img.size * 100
    return mask_percentage

def filter_green_channel(np_img, green_thresh=200, avoid_overmask=True, overmask_thresh=90, output_type="bool"):
    log.debug('Filtering green channel. Output type: {}'.format(output_type))
    g = np_img[:, :, 1]
    gr_ch_mask = (g < green_thresh) & (g > 0)
    mask_percentage = mask_percent(gr_ch_mask)
    if (mask_percentage >= overmask_thresh) and (green_thresh < 255) and (avoid_overmask is True):
        new_green_thresh = math.ceil((255 - green_thresh) / 2 + green_thresh)
        #print(
        #  "Mask percentage %3.2f%% >= overmask threshold %3.2f%% for Remove Green Channel green_thresh=%d, so try %d" % (
        #    mask_percentage, overmask_thresh, green_thresh, new_green_thresh))
        gr_ch_mask = filter_green_channel(np_img, new_green_thresh, avoid_overmask, overmask_thresh, output_type)
    np_img = gr_ch_mask

    if output_type == "bool":
        pass
    elif output_type == "float":
        np_img = np_img.astype(float)
    else:
        np_img = np_img.astype("uint8") * 255

    return np_img

def filter_grays(rgb, tolerance=15, output_type="bool"):
    log.debug('Filtering grays. Output type: {}'.format(output_type))
    (h, w, c) = rgb.shape

    rgb = rgb.astype(np.int)
    rg_diff = abs(rgb[:, :, 0] - rgb[:, :, 1]) <= tolerance
    rb_diff = abs(rgb[:, :, 0] - rgb[:, :, 2]) <= tolerance
    gb_diff = abs(rgb[:, :, 1] - rgb[:, :, 2]) <= tolerance
    result = ~(rg_diff & rb_diff & gb_diff)

    if output_type == "bool":
        pass
    elif output_type == "float":
        result = result.astype(float)
    else:
        result = result.astype("uint8") * 255
    return result

def filter_red_pen(rgb, output_type="bool"):
    log.debug('Filter red pen. Output type: {}'.format(output_type))
    result = filter_red(rgb, red_lower_thresh=150, green_upper_thresh=80, blue_upper_thresh=90) & \
              filter_red(rgb, red_lower_thresh=110, green_upper_thresh=20, blue_upper_thresh=30) & \
              filter_red(rgb, red_lower_thresh=185, green_upper_thresh=65, blue_upper_thresh=105) & \
              filter_red(rgb, red_lower_thresh=195, green_upper_thresh=85, blue_upper_thresh=125) & \
              filter_red(rgb, red_lower_thresh=220, green_upper_thresh=115, blue_upper_thresh=145) & \
              filter_red(rgb, red_lower_thresh=125, green_upper_thresh=40, blue_upper_thresh=70) & \
              filter_red(rgb, red_lower_thresh=200, green_upper_thresh=120, blue_upper_thresh=150) & \
              filter_red(rgb, red_lower_thresh=100, green_upper_thresh=50, blue_upper_thresh=65) & \
              filter_red(rgb, red_lower_thresh=85, green_upper_thresh=25, blue_upper_thresh=45)
    if output_type == "bool":
          pass
    elif output_type == "float":
          result = result.astype(float)
    else:
          result = result.astype("uint8") * 255
    return result

def filter_red(rgb, red_lower_thresh, green_upper_thresh, blue_upper_thresh, output_type="bool",
               display_np_info=False):
    log.debug('Filter red. Output type: {}'.format(output_type))
    r = rgb[:, :, 0] > red_lower_thresh
    g = rgb[:, :, 1] < green_upper_thresh
    b = rgb[:, :, 2] < blue_upper_thresh
    result = ~(r & g & b)
    if output_type == "bool":
          pass
    elif output_type == "float":
          result = result.astype(float)
    else:
          result = result.astype("uint8") * 255
    return result

def filter_green(rgb, red_upper_thresh, green_lower_thresh, blue_lower_thresh, output_type="bool",
                 display_np_info=False):
    """
    Create a mask to filter out greenish colors, where the mask is based on a pixel being below a
    red channel threshold value, above a green channel threshold value, and above a blue channel threshold value.
    Note that for the green ink, the green and blue channels tend to track together, so we use a blue channel
    lower threshold value rather than a blue channel upper threshold value.

    Args:
      rgb: RGB image as a NumPy array.
      red_upper_thresh: Red channel upper threshold value.
      green_lower_thresh: Green channel lower threshold value.
      blue_lower_thresh: Blue channel lower threshold value.
      output_type: Type of array to return (bool, float, or uint8).
      display_np_info: If True, display NumPy array info and filter time.

    Returns:
      NumPy array representing the mask.
    """
    log.debug('Filter green. Output type: {}'.format(output_type))
    r = rgb[:, :, 0] < red_upper_thresh
    g = rgb[:, :, 1] > green_lower_thresh
    b = rgb[:, :, 2] > blue_lower_thresh
    result = ~(r & g & b)
    if output_type == "bool":
          pass
    elif output_type == "float":
          result = result.astype(float)
    else:
          result = result.astype("uint8") * 255
    return result


def filter_green_pen(rgb, output_type="bool"):
    """
    Create a mask to filter out green pen marks from a slide.

    Args:
      rgb: RGB image as a NumPy array.
      output_type: Type of array to return (bool, float, or uint8).

    Returns:
      NumPy array representing the mask.
    """
    log.debug('Filter green pen. Output type: {}'.format(output_type))
    result = filter_green(rgb, red_upper_thresh=150, green_lower_thresh=160, blue_lower_thresh=140) & \
            filter_green(rgb, red_upper_thresh=70, green_lower_thresh=110, blue_lower_thresh=110) & \
            filter_green(rgb, red_upper_thresh=45, green_lower_thresh=115, blue_lower_thresh=100) & \
            filter_green(rgb, red_upper_thresh=30, green_lower_thresh=75, blue_lower_thresh=60) & \
            filter_green(rgb, red_upper_thresh=195, green_lower_thresh=220, blue_lower_thresh=210) & \
            filter_green(rgb, red_upper_thresh=225, green_lower_thresh=230, blue_lower_thresh=225) & \
            filter_green(rgb, red_upper_thresh=170, green_lower_thresh=210, blue_lower_thresh=200) & \
            filter_green(rgb, red_upper_thresh=20, green_lower_thresh=30, blue_lower_thresh=20) & \
            filter_green(rgb, red_upper_thresh=50, green_lower_thresh=60, blue_lower_thresh=40) & \
            filter_green(rgb, red_upper_thresh=30, green_lower_thresh=50, blue_lower_thresh=35) & \
            filter_green(rgb, red_upper_thresh=65, green_lower_thresh=70, blue_lower_thresh=60) & \
            filter_green(rgb, red_upper_thresh=100, green_lower_thresh=110, blue_lower_thresh=105) & \
            filter_green(rgb, red_upper_thresh=165, green_lower_thresh=180, blue_lower_thresh=180) & \
            filter_green(rgb, red_upper_thresh=140, green_lower_thresh=140, blue_lower_thresh=150) & \
            filter_green(rgb, red_upper_thresh=185, green_lower_thresh=195, blue_lower_thresh=195)
    if output_type == "bool":
          pass
    elif output_type == "float":
          result = result.astype(float)
    else:
          result = result.astype("uint8") * 255
    return result

def filter_blue(rgb, red_upper_thresh, green_upper_thresh, blue_lower_thresh, output_type="bool",
                display_np_info=False):
    log.debug('Filter blue. Output type: {}'.format(output_type))
    r = rgb[:, :, 0] < red_upper_thresh
    g = rgb[:, :, 1] < green_upper_thresh
    b = rgb[:, :, 2] > blue_lower_thresh
    result = ~(r & g & b)
    if output_type == "bool":
          pass
    elif output_type == "float":
          result = result.astype(float)
    else:
          result = result.astype("uint8") * 255
    return result


def filter_blue_pen(rgb, output_type="bool"):
    """
    Create a mask to filter out blue pen marks from a slide.

    Args:
      rgb: RGB image as a NumPy array.
      output_type: Type of array to return (bool, float, or uint8).

    Returns:
      NumPy array representing the mask.
    """
    log.debug('Filter blue pen. Output type: {}'.format(output_type))
    result = filter_blue(rgb, red_upper_thresh=60, green_upper_thresh=120, blue_lower_thresh=190) & \
            filter_blue(rgb, red_upper_thresh=120, green_upper_thresh=170, blue_lower_thresh=200) & \
            filter_blue(rgb, red_upper_thresh=175, green_upper_thresh=210, blue_lower_thresh=230) & \
            filter_blue(rgb, red_upper_thresh=145, green_upper_thresh=180, blue_lower_thresh=210) & \
            filter_blue(rgb, red_upper_thresh=37, green_upper_thresh=95, blue_lower_thresh=160) & \
            filter_blue(rgb, red_upper_thresh=30, green_upper_thresh=65, blue_lower_thresh=130) & \
            filter_blue(rgb, red_upper_thresh=130, green_upper_thresh=155, blue_lower_thresh=180) & \
            filter_blue(rgb, red_upper_thresh=40, green_upper_thresh=35, blue_lower_thresh=85) & \
            filter_blue(rgb, red_upper_thresh=30, green_upper_thresh=20, blue_lower_thresh=65) & \
            filter_blue(rgb, red_upper_thresh=90, green_upper_thresh=90, blue_lower_thresh=140) & \
            filter_blue(rgb, red_upper_thresh=60, green_upper_thresh=60, blue_lower_thresh=120) & \
            filter_blue(rgb, red_upper_thresh=110, green_upper_thresh=110, blue_lower_thresh=175)
    if output_type == "bool":
          pass
    elif output_type == "float":
          result = result.astype(float)
    else:
          result = result.astype("uint8") * 255
    return result

def filter_remove_small_objects(np_img, min_size=3000, avoid_overmask=True, overmask_thresh=95, output_type="uint8"):
    log.debug('Filter remove small objects. Output type: {}'.format(output_type))
    rem_sm = np_img.astype(bool)  # make sure mask is boolean
    rem_sm = sk_morphology.remove_small_objects(rem_sm, min_size=min_size)
    mask_percentage = mask_percent(rem_sm)
    if (mask_percentage >= overmask_thresh) and (min_size >= 1) and (avoid_overmask is True):
        new_min_size = min_size / 2
        #print("Mask percentage %3.2f%% >= overmask threshold %3.2f%% for Remove Small Objs size %d, so try %d" % (
        #    mask_percentage, overmask_thresh, min_size, new_min_size))
        rem_sm = filter_remove_small_objects(np_img, new_min_size, avoid_overmask, overmask_thresh, output_type)
    np_img = rem_sm

    if output_type == "bool":
          pass
    elif output_type == "float":
          np_img = np_img.astype(float)
    else:
          np_img = np_img.astype("uint8") * 255

    return np_img      

In [9]:
def read_polygons(annotation_filepath, scale_factor):
    """
        Utility function to read an annotation XML file and create a list of vertices for
        polygon delimiting the cancerous area.
    """
    tumor_keywords = ['Metastasis', 'Carcinoma', 'Tumor', 'Rough tumor']
    # Read cancer polygon area
    polygons = []
    root = ET.parse(str(annotation_filepath)).getroot()
    for anno_tag in root.findall('Annotations/Annotation'):
        polygon = []
        
        if anno_tag.get('PartOfGroup') not in tumor_keywords:
            continue

        for coord in anno_tag.findall('Coordinates/Coordinate'):
            polygon.append((float(coord.get('X')) / scale_factor, float(coord.get('Y')) / scale_factor))
        polygons.append(polygon)
    return polygons

def create_annotation_mask(annot_file, slide, level=0, scaling_factor=1):
    """
        Creates a binary mask for the cancer area (white) from annotation file
    """
    log.debug('Creating annotation mask.')
    label_mask = Image.new('L', size=slide.level_dimensions[level], color='BLACK')
    label_draw = ImageDraw.Draw(label_mask)
    polygons = read_polygons(annot_file, scaling_factor)
    for polygon in polygons:
        label_draw.polygon(xy=polygon, outline=(255), fill=(255))
    return label_mask

def filter_background(np_img):
    """
        Applies filters on the numpy image to filter background
    """
    log.debug('Filtering background.')
    mask_not_green = filter_green_channel(np_img)
    mask_not_gray = filter_grays(np_img) 
    mask_gray_green = mask_not_gray & mask_not_green  
    mask_remove_small = filter_remove_small_objects(mask_gray_green, min_size=500, output_type="bool") 
    return Image.fromarray(mask_remove_small.astype(np.uint8))

def tissue_percent(tile_mask, tile_size_mini):
    """
        Calculates the fraction of non-black area vs total area in a mask
    """
    ts_count = np.count_nonzero(tile_mask)
    bg_count = tile_size_mini*tile_size_mini
    return ts_count / bg_count

In [10]:
def calculate_progress(y_coord, x_coord, slide_width, slide_height, center_size):
    """
        Calculate progress as a ratio of processed slide area vs total slide area
    """
    total = slide_height*slide_width
    processed = (y_coord*slide_width + center_size*x_coord)
    percentage = int(processed / total * 100)
    return percentage

In [17]:
def process_wsi(slide_fn, annot_fn, center_size, margin_size, level=0, downsample=40, background_threshold=0.2,
               cancer_threshold=1):
    """
        Process each tile from WSI
        
        1) Get handler for WSI slide
        2) Create black & white annotation mask (WSI size)
        3) Create black & white background mask (downsized)
        4) For each column and row
            4.1) Retrieve tile from scaled down background mask
            4.2) If tile is NOT background, retrieve center of the tile from the annotation mask
            4.3) Label and add it to the table
    """
    percent = -1
    
    tile_size = center_size + 2*margin_size
    
    print('Opening original slide')
    slide = os.open_slide(slide_fn)
    slide_width, slide_height = slide.dimensions
    log.debug('Original slide dimensions: (w:{}, h:{})'.format(slide_width, slide_height))
    
    scaling_factor = slide.level_downsamples[level]
    log.debug('Sampling level: {}'.format(level))
    log.debug('Sampling factor: {}'.format(scaling_factor))
    
    downsample_level = slide.get_best_level_for_downsample(downsample)
    downsample_factor = slide.level_downsamples[downsample_level]
    log.debug('Downsample level: {}'.format(downsample_level))
    log.debug('Downsample factor: {}'.format(downsample_factor))
    log.debug('Downsample dimensions: {}'.format(slide.level_dimensions[downsample_level]))
    
    scale = lambda val: int(val // downsample_factor)

    log.info('Initializing offset map')
    offset_map = {'row': [], 'col': [], 'class': []}

    annotation_mask = create_annotation_mask(annot_fn, slide, level=level, scaling_factor=scaling_factor)

    mini_slide = slide.read_region(location=(0,0), level=downsample_level, size=slide.level_dimensions[downsample_level])
    background_mask = filter_background(np.array(mini_slide))    

    log.info('Processing WSI tiles.')
    for row, y_coord in enumerate(range(0,slide_height,center_size), 1):
        for col, x_coord in enumerate(range(0,slide_width,center_size), 1):

            # CALCULATE PROGRESS
            new_percent = calculate_progress(y_coord, x_coord, slide_width, slide_height, center_size)
            if new_percent > percent:
                percent = new_percent
                print('Progress: {:3}%\r'.format(percent), end='')
            
            # RETRIEVE TILE FROM BACKGROUND MASK
            tile_mask = background_mask.crop((scale(x_coord), scale(y_coord), 
                                              scale(x_coord+tile_size), scale(y_coord+tile_size)))

            # SKIP IF TILE IS BACKGROUND
            if tissue_percent(np.array(tile_mask), scale(tile_size)) < background_threshold:
                continue

            # RETRIEVE CENTER REGION OF A TILE FROM CANCER MASK
            tile_mask = annotation_mask.crop((x_coord+margin_size, y_coord+margin_size, 
                                              x_coord+margin_size+center_size, y_coord+margin_size+center_size))

            # TILE IS CANCER IF AT LEAST 1 PIXEL IN CENTER IS CANCEROUS
            class_ = 0
            if np.count_nonzero(np.array(tile_mask)) >= cancer_threshold:
                class_ = 1

            offset_map['row'].append(row)
            offset_map['col'].append(col)
            offset_map['class'].append(class_)
            
            
    offset_df = pd.DataFrame.from_dict(offset_map)
    offset_df.set_index(['col', 'row'], inplace=True)
    offset_df['weight'] = get_weight_vector(offset_df)
    return offset_df

In [8]:
def get_weight_vector(df):
    log.debug('Calculating weight vector.')
    df.sort_values(inplace=True, by='class')
    c = Counter(df['class'].values)
    weights_normal = ([0.5 / c[0]] * c[0]) if c[0] > 0 else []
    weights_cancer = ([0.5 / c[1]] * c[1]) if c[1] > 0 else []
    return weights_normal + weights_cancer

In [20]:
from multiprocessing import Pool

def worker_task(slide_fn, annot_fn, output_fn, center_size, margin_size, level, downsample, background_threshold, cancer_threshold):
    try:
        log.info(slide_fn)
        offset_df = process_wsi(slide_fn=slide_fn, annot_fn=annot_fn, 
                                center_size=center_size, margin_size=margin_size, 
                                level=level, downsample=downsample, background_threshold=0.2,
                                cancer_threshold=1)
        log.info('Retrieved {} tiles.'.format(len(offset_df)))
        offset_df.to_pickle(output_fn, compression='gzip')
        return True
    except:
        log.error('Exception occurred for slide: {}'.format(slide_fn))
        return False

def main():
    log = logging.getLogger()
    log = prepare_logger(log, level=logging.INFO, filename_template=None)

    level = 0
    downsample = 32
    center_size = 32
    margin_size = 32
    tile_size = center_size + 2*margin_size

    DATASET_NAME = 'Mammy'

    MRXS_DATASET = '/mnt/data/scans/AI scans/{ds_name}/'.format(ds_name=DATASET_NAME)
    ANNOT_DIR = '/home/matejg/rough/'
    OUTPUT_DIR = Path('/home/matejg/wsi_maps/Mammy/level{level}/normal/'.format(level=level))

    if not OUTPUT_DIR.exists():
        log.debug('Creating output directory.')
        OUTPUT_DIR.mkdir(parents=True)

    with Pool(processes=None) as pool:
        
        for slide_fn in Path(MRXS_DATASET).glob('*.mrxs'):
            annot_fn = (Path(ANNOT_DIR) / slide_fn.name).with_suffix('.xml')
            output_fn = OUTPUT_DIR / '{slide_name}-level{level}-ds{downsample}-tile{tile_size}-center{center_size}.gz'.format(slide_name=slide_fn.stem, level=level, downsample=downsample, tile_size=tile_size, center_size=center_size)

            if annot_fn.exists():
                pool.apply_async(worker_task, (str(slide_fn), str(annot_fn), str(output_fn), center_size, margin_size, level, downsample, background_threshold, cancer_threshold))
            else:
                log.warning('{} skipped. No annotation XML found.'.format(slide_fn.stem))

SyntaxError: invalid syntax (<ipython-input-20-31aafe1d1a40>, line 23)

In [ ]:
if __name__=='__main__':
    main()

In [6]:
""" CREATE CONTEXT PD's """
def test_get_context_tiles(pd_data, tile_coord, context_size):
    assert context_size % 2 == 1, 'context size must be and odd number'
    context_range = int((context_size - 1) / 2)
    
    context_tile_coords = []
    for x in range(-context_range, context_range+1):
        for y in range(-context_range, context_range+1):
            context_tile_coords.append((tile_coord[0]+x,tile_coord[1]+y) in pd_data.index)
    return np.all(context_tile_coords)

def generate_context(col, row):
    col_rows = []
    for x in range(-1,2):
        for y in range(-1,2):
            col_rows.append((col+x, row+y))
    return col_rows

def filter_macro_tiles():
    DIR_PATH = Path('/home/matejg/Project/crc_ml/data/processed/Prostata/level1/')
    IN_DIR = DIR_PATH / 'tiles'
    OUT_DIR = DIR_PATH / 'macrotiles'
    
    if not OUT_DIR.exists():
        OUT_DIR.mkdir(parents=True)
        
    slides = list(IN_DIR.glob('*.gz'))
    CONTEXT_SIZE = 3
    
    for slide_num, _fn in enumerate(slides):
        print('{}/{}'.format(slide_num, len(slides)))
        _pd_data = pd.read_pickle(str(_fn))
        boolean_mask = []
        for row_idx, row in enumerate(_pd_data.iterrows()):
            if row_idx % 5000 == 0:
                print('{:,}/{:,}\r'.format(row_idx, len(_pd_data)), end='')
            boolean_mask.append(test_get_context_tiles(_pd_data, row[0], CONTEXT_SIZE))
        _macro_tile_pd = _pd_data[boolean_mask]
        print('{} | {} -> {} ({:.2%} reduction.)'.format(_fn.stem, len(_pd_data), len(_macro_tile_pd), len(_macro_tile_pd)/len(_pd_data)))
        _macro_tile_pd.to_pickle(str(OUT_DIR / '{}-context{}.gz'.format(_fn.stem, CONTEXT_SIZE)))
        
filter_macro_tiles()    

0/133
P-2019_1399-02-0-bg4-tile96 | 51862 -> 41698 (80.40% reduction.)
1/133
P-2019_3204-08-0-bg4-tile96 | 9078 -> 6255 (68.90% reduction.)
2/133
P-2019_4624-07-0-bg4-tile96 | 10817 -> 7365 (68.09% reduction.)
3/133
P-2019_3126-08-0-bg4-tile96 | 8319 -> 5477 (65.84% reduction.)
4/133
P-2019_4336-11-1-bg4-tile96 | 18311 -> 13278 (72.51% reduction.)
5/133
P-2019_2587-01-0-bg4-tile96 | 8457 -> 5736 (67.83% reduction.)
6/133
P-2019_2587-11-0-bg4-tile96 | 9572 -> 6283 (65.64% reduction.)
7/133
P-2019_2216-03-1-bg4-tile96 | 15184 -> 10376 (68.34% reduction.)
8/133
P-2019_3124-14-0-bg4-tile96 | 12007 -> 8817 (73.43% reduction.)
9/133
P-2019_3672-07-1-bg4-tile96 | 5192 -> 4194 (80.78% reduction.)
10/133
P-2019_4095-19-0-bg4-tile96 | 13452 -> 10108 (75.14% reduction.)
11/133
P-2019_2552-10-0-bg4-tile96 | 11368 -> 8125 (71.47% reduction.)
12/133
P-2019_3292-04-1-bg4-tile96 | 13504 -> 9369 (69.38% reduction.)
13/133
P-2019_4624-14-0-bg4-tile96 | 15960 -> 11858 (74.30% reduction.)
14/133
P-2019_31

  0 | /home/matejg/wsi_maps/Prostata/level1/normal/P-2019_1399-02-0-bg4-tile96.gz
  1 | /home/matejg/wsi_maps/Prostata/level1/normal/P-2019_3204-08-0-bg4-tile96.gz
  2 | /home/matejg/wsi_maps/Prostata/level1/normal/P-2019_4624-07-0-bg4-tile96.gz
  3 | /home/matejg/wsi_maps/Prostata/level1/normal/P-2019_3126-08-0-bg4-tile96.gz
  4 | /home/matejg/wsi_maps/Prostata/level1/normal/P-2019_4336-11-1-bg4-tile96.gz
  5 | /home/matejg/wsi_maps/Prostata/level1/normal/P-2019_2587-01-0-bg4-tile96.gz
  6 | /home/matejg/wsi_maps/Prostata/level1/normal/P-2019_2587-11-0-bg4-tile96.gz
  7 | /home/matejg/wsi_maps/Prostata/level1/normal/P-2019_2216-03-1-bg4-tile96.gz
  8 | /home/matejg/wsi_maps/Prostata/level1/normal/P-2019_3124-14-0-bg4-tile96.gz
  9 | /home/matejg/wsi_maps/Prostata/level1/normal/P-2019_3672-07-1-bg4-tile96.gz
 10 | /home/matejg/wsi_maps/Prostata/level1/normal/P-2019_4095-19-0-bg4-tile96.gz
 11 | /home/matejg/wsi_maps/Prostata/level1/normal/P-2019_2552-10-0-bg4-tile96.gz
 12 | /home/mate